In [1]:
from nltk.tokenize import word_tokenize
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import nltk
from textblob import TextBlob
import json
from tqdm import tqdm
from nltk.corpus import twitter_samples
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
import string
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
#from keras.preprocessing.text import Tokenizer

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens_index in range(0,len(cleaned_tokens_list)):
        return dict([token, True] for token in cleaned_tokens_list[tweet_tokens_index]) 
    
    
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

stop_words = stopwords.words('english')
data_ref=pd.read_excel('tweets.xlsx') 

string_vec=string.punctuation+"--"+'...'

def remove_noise(tokens):
    
    
    cleaned_tokens = []
    for token, tag in pos_tag(tokens):
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)
        
        if len(token) > 0 and (token not in string_vec) and (token.lower() not in stop_words):
                cleaned_tokens.append(token)
    return cleaned_tokens



import csv


def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def preprocess_text2(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','', text)
    text = re.sub('@[^\s]+','', text)
    #text = text.lower().replace("ё", "е")
    text = text.replace("ё", "е")
    dat_str=data_ref.iloc[0]['full_text']
    text = re.sub(dat_str[-10:-7],'\'', text)
    #text = re.sub('[^a-zA-Zа-яА-Я1-9]+',' ', text)
    
    text = re.sub(' +',' ', text)
    text = re.sub(r' &amp','\'', text)
    text = re.sub(r'\r','', text)
    text = re.sub(r'\n','', text)
    text = re.sub('[$|@|#|%|^|&|*|\(|\)|\\|\"|\"]',' ', text)
    text=deEmojify(text)
    #print(text)
    #text = re.sub('[^a-zA-Zа-яА-Я1-9]+',' ', text)
    #text = re.sub(' +',' ', text)
    return text.strip()



data=pd.read_csv('vote_all (1).csv',encoding = "ISO-8859–1")

data.head()



,Tweet ID,text,SentiStrength,Vader,Textblob,Vote
0,1278320583718178816,Business Group Complains Trump H-1B Reform Boo...,Negative,Negative,Negative,Negative
1,1278310669885026305,"""Who's the absent candidate now?...'Sleepy Joe...",Neutral,Negative,Positive,Neutral
2,1278346691171586049,That moron trump vows to veto the Defense Bill...,Neutral,Negative,Negative,Negative
3,1278368973948694528,@ClareTyne @mesainy @HKrassenstein @realDonald...,Negative,Positive,Neutral,Neutral
4,1278303504071905282,"1. Funny how Biden barely criticizes Putin, an...",Neutral,Negative,Positive,Neutral


In [2]:

def preprocess_text2(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','', str(text))
    text = re.sub('@[^\s]+','', text)
    #text = text.lower().replace("ё", "е")
    text = text.replace("ё", "е")
    dat_str=data_ref.iloc[0]['full_text']
    text = re.sub(dat_str[-10:-7],'\'', text)
    #text = re.sub('[^a-zA-Zа-яА-Я1-9]+',' ', text)
    
    text = re.sub(' +',' ', text)
    text = re.sub(r' &amp','\'', text)
    text = re.sub(r'\r','', text)
    text = re.sub(r'\n','', text)
    text = re.sub('[$|@|#|%|^|&|*|\(|\)|\\|\"|\"]',' ', text)
    text=deEmojify(text)
    #print(text)
    #text = re.sub('[^a-zA-Zа-яА-Я1-9]+',' ', text)
    #text = re.sub(' +',' ', text)
    return text.strip()


data["processed_text"] =  data['text'].apply(preprocess_text2)
pd.options.display.max_colwidth = 150
data.head()


,Tweet ID,text,SentiStrength,Vader,Textblob,Vote,processed_text
0,1278320583718178816,Business Group Complains Trump H-1B Reform Boosting U.S. Graduates. Big tech is whining that they will have to hire American instead of cheap fore...,Negative,Negative,Negative,Negative,Business Group Complains Trump H-1B Reform Boosting U.S. Graduates. Big tech is whining that they will have to hire American instead of cheap fore...
1,1278310669885026305,"""Who's the absent candidate now?...'Sleepy Joe' is signaling that he's very much awake -- and dialed into a moment where Trump's leadership is rip...",Neutral,Negative,Positive,Neutral,Who's the absent candidate now?...'Sleepy Joe' is signaling that he's very much awake -- and dialed into a moment where Trump's leadership is ripe...
2,1278346691171586049,"That moron trump vows to veto the Defense Bill if it includes renaming bases. So once again, military salaries and defense preparedness are second...",Neutral,Negative,Negative,Negative,"That moron trump vows to veto the Defense Bill if it includes renaming bases. So once again, military salaries and defense preparedness are second..."
3,1278368973948694528,@ClareTyne @mesainy @HKrassenstein @realDonaldTrump @NYCMayor @JoeBiden Yep. Torturing that guy because the Dems donât care.,Negative,Positive,Neutral,Neutral,Yep. Torturing that guy because the Dems donât care.
4,1278303504071905282,"1. Funny how Biden barely criticizes Putin, and on the China virus doesnât criticize Xi.Â Instead, he smears Trump.Â The reason is the Democra...",Neutral,Negative,Positive,Neutral,"1. Funny how Biden barely criticizes Putin, and on the China virus doesnât criticize Xi.Â Instead, he smears Trump.Â The reason is the Democra..."


In [3]:
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = nltk.word_tokenize)
#cv=CountVectorizer(stop_words='english')
text_counts = cv.fit_transform(data['processed_text'])

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

X_train, X_test, Y_train, Y_test = train_test_split(text_counts,data['Vote'], test_size=0.25, random_state=5)



In [5]:
#num_features=len(X_train.columns)
print(X_train.shape)

(2097669, 871563)


In [6]:

def replace_with_array(text):
    if text=='\tNegative'or text=='Negative':
        array=1
    elif text=='\tPositive'or text=='Positive':
        array=0
    else:
        array=2
    return array
Y_train.head()            
Y_train["Vote_num"] =  Y_train.apply(replace_with_array)
Y_test["Vote_num"] =  Y_test.apply(replace_with_array)


#print(X_test[20].shape)
#print(X_train.shape[0])


In [7]:
Y_train.head()         

622754     Neutral
1864131    Neutral
2642573    Neutral
2086943    Neutral
2315382    Neutral
Name: Vote, dtype: object

In [23]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler



#tSVD = TruncatedSVD(n_components=1000)

#Y_train=Y_train["Vote_num"]
#Y_test=Y_test["Vote_num"]

X_tr=X_train[0:100000,:]
Y_tr=Y_train[0:100000]

#scaler=MinMaxScaler()

#X_trNew=scaler.fit_transform(X_tr)


#X_tr_tSVD=tSVD.fit_transform(X_tr)



In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=1000,criterion='gini',warm_start=True)
#for index in range(10):
clf.fit(X_tr,Y_tr)
    

In [21]:
print(X_test.shape)
print(Y_test.shape)


(699223, 871563)
(699223,)


In [ ]:
#X_tsNew=scaler.transform(X_test)

print('Accuracy is ',clf.score(X_test,Y_test))


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Y_test, clf.predict(X_test)))


In [21]:
from sklearn.metrics import confusion_matrix

confusion_matrix(Y_test, clf.predict(X_test),labels=['Negative','Neutral','Positive'])

array([[116909,  69924,   5253],
       [ 18374, 310771,  12528],
       [  6887,  69530,  89047]])

In [25]:
print(X_test)

  (0, 8177)	1
  (0, 11932)	3
  (0, 194726)	1
  (0, 205133)	1
  (0, 239119)	1
  (0, 259682)	1
  (0, 367006)	1
  (0, 394698)	1
  (0, 423396)	1
  (0, 579016)	2
  (0, 593743)	1
  (0, 641373)	1
  (0, 644949)	1
  (0, 659349)	1
  (1, 8177)	2
  (1, 11932)	2
  (1, 255139)	1
  (1, 292709)	1
  (1, 314191)	1
  (1, 336768)	2
  (1, 442342)	1
  (1, 444685)	1
  (1, 526976)	1
  (1, 533085)	1
  (1, 613192)	1
  :	:
  (699219, 742367)	1
  (699219, 742615)	2
  (699219, 780517)	1
  (699219, 793096)	1
  (699220, 207945)	1
  (699220, 460291)	1
  (699220, 492394)	1
  (699220, 506702)	1
  (699221, 11932)	1
  (699221, 64904)	1
  (699221, 106192)	1
  (699221, 109607)	1
  (699221, 201880)	1
  (699221, 216342)	1
  (699221, 366458)	1
  (699221, 409352)	1
  (699221, 490796)	1
  (699221, 576677)	1
  (699221, 591034)	1
  (699221, 691705)	1
  (699221, 744565)	1
  (699222, 0)	3
  (699222, 11932)	1
  (699222, 124247)	1
  (699222, 580729)	1


In [19]:
clf=RandomForestClassifier(n_estimators=80,criterion='entropy').fit(X_tr,Y_tr)


print('Accuracy is ',clf.score(X_test,Y_test))
            

Accuracy is  0.728899077976554


In [ ]:
X_ts=X_test[0:100000,:]
Y_ts=Y_test[0:100000]
print('done')
parameters = {'criterion':('gini', 'entropy'), 'n_estimators':[60,80,100,120]}
RFC = RandomForestClassifier()
clf_Grid=GridSearchCV(RFC, parameters).fit(X_tr,Y_tr)
print('Best params for Random forest are ',clf_Grid.best_params_)

RFC.set_params(criterion=clf_Grid.best_params_['criterion'], n_estimators= clf_Grid.best_params_['n_estimators'])
RFC.fit(X_tr,y_tr)

print('Accuracy is ',RFC.score(X_ts,Y_ts))

done


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true, RFC.predict(X_test)))

In [ ]:
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = nltk.word_tokenize)
#cv=CountVectorizer(stop_words='english')
text_counts = cv.fit_transform(data[0:700000]['processed_text'])

X_train, X_testNew, Y_train, Y_testNew = train_test_split(text_counts,data[0:700000]['Vote']\
                                                          , test_size=0.25, random_state=5)

def replace_with_array(text):
    if text=='\tNegative'or text=='Negative':
        array=1
    elif text=='\tPositive'or text=='Positive':
        array=0
    else:
        array=2
    return array
Y_train.head()            
Y_train["Vote_num"] =  Y_train.apply(replace_with_array)
Y_testNew["Vote_num"] =  Y_testNew.apply(replace_with_array)

X_tr=X_train[0:100000,:]
Y_tr=Y_train[0:100000]

clf=RandomForestClassifier(n_estimators=100,criterion='gini').fit(X_tr,Y_tr)

print('Accuracy is ',clf.score(X_testNew,Y_testNew))
print(classification_report(y_true, clf.predict(X_testNew)))

In [ ]:
clf=RandomForestClassifier(n_estimators=15,criterion='entropy').fit(X_train,Y_train)
print('Accuracy is ',clf.score(X_test,Y_test))

In [ ]:
#import xgb
from scipy.sparse import csr_matrix
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

tSVD = TruncatedSVD(n_components=10000)
X_tr_tSVD=tSVD.fit_transform(X_train)
X_ts_tSVD=tSVD.transform(X_test)

clf=RandomForestClassifier(n_estimators=10).fit(X_tr_tSVD,Y_train)
print('Accuracy is ',clf.score(X_ts_tSVD,Y_test))



In [ ]:
print('lala')

In [ ]:
"""
error=0
for index in range(int(X_test.shape[0])):
    x=X_test[index]
    x=x.todense()
    y=Y_ts_tensor[index]
    #x=torch.from_numpy(x)
    x=torch.Tensor(x)
    y_predict=net_model.net(x)
    error=error+(y_predict!=y)

print(error)
"""
    

In [ ]:

#print(Y_train.iloc[0])


In [ ]:
"""

import torch.nn.functional as F
import torch
class Net(torch.nn.Module):
        def __init__(self, n_feature, n_hidden=100,n_hidden12=100, n_output=3):
                super(Net, self).__init__()
                self.hidden1 = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
                
                self.hidden12=torch.nn.Linear(n_hidden,n_hidden12)
                self.predict = torch.nn.Linear(n_hidden12, n_output)   # output layer

        def net(self, x):
                x = F.relu(self.hidden1(x))      # activation function for hidden layer
                
                x=torch.tanh(self.hidden12(x))
                x=self.predict(x)
                x=F.softmax(x,dim=1)          # linear output
                return x
            
class LSTM(nn.Module):
        def __init__(self, input_shape=458750, n_actions=3):
                super(LSTM, self).__init__()
                self.lstm = nn.LSTM(input_shape, 100)
                self.hidden2tag = nn.Linear(100, n_actions)

        def net(self, x):
                out1,_ = self.lstm(x)
                out1 = F.tanh(self.hidden2tag(out1))
                out1=torch.max(F.softmax(out1))

                return out1
            
net_model=Net(n_feature=int(X_train.shape[1]))
optimizer = torch.optim.Adam(net_model.parameters(), lr=0.01)
print('start')
Y_tr_tensor = (Y_train["Sentiment_array"].values)
Y_ts_tensor = (Y_test["Sentiment_array"].values)

for index in range(int(X_train.shape[0])): 
    optimizer.zero_grad()
    print(index)
    x=X_train[index]
    
    y=torch.tensor(Y_tr_tensor[index])
    loss_func = torch.nn.CrossEntropyLoss()
    net_model.zero_grad()
    
    x=x.todense()
    
    x=torch.Tensor(x)
    
    
   
    
    net_out=net_model.net(x)
    
    y=y.unsqueeze(0)
    
    loss = loss_func(net_out,y)
    loss.backward()
    optimizer.step()
    print(loss)

count=0
with open('dataset_filt.jsonl','r') as f:
        all_tokens_list=[]
        for line in f:
            count=count+1
            tweet = json.loads(line)
            text0=tweet["full_text"]
            text1= preprocess_text2(tweet["full_text"]).lower()
            
            tokenized_text=word_tokenize(text1)
            
            all_tokens_list=all_tokens_list+(remove_noise(tokenized_text))
            print(count)
            
            
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = nltk.word_tokenize)
#cv=CountVectorizer(stop_words='english')
text_counts = cv.fit_transform(data['processed_text'])

#print(all_tokens_list)
counts = Counter(all_tokens_list)
#print(counts)
#print(counts,'\n')
vocab = sorted(counts,  key=counts.get,reverse=True)
#print(vocab,'\n')
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
#print(vocab_to_int)

tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(remove_noise(tokenized_text))


print(all_tokens_list[3])
all_pos_words = get_tweets_for_model(all_tokens_list)

print(all_pos_words)

print(remove_noise(tokenized_text))

counts=Counter(remove_noise(tokenized_text))
print(counts)
max_fatures = 2000

X = tokenizer.texts_to_sequences(['How','are','you'])
vocab = sorted(counts, key=counts.get, reverse=True)
#print(vocab)
print(X)


"""